In [2]:
!pip install nbformat --upgrade

In [1]:
import os
import json
import pandas as pd
import psycopg2
import streamlit as st
from streamlit_option_menu import option_menu
import matplotlib.pyplot as plt
import plotly.express as px
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import requests

import os
import json
import pandas as pd
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

2025-08-12 21:34:03.006 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
connect = psycopg2.connect(
     host = "localhost",
     user = 'postgres',
     port = '5432',
     password = '01234',
     database = 'phonepy_transaction'
)

connect.set_isolation_level (ISOLATION_LEVEL_AUTOCOMMIT)

cursor = connect.cursor()

#Agg_insurance

cursor.execute ("select * from agg_insurance")
connect.commit()
tabel1 = cursor.fetchall()

Agg_insurance = pd.DataFrame(tabel1, columns=('State', 'Year', 'Quarter', 'Transaction_Name', 'Transaction_count', 'Transaction_amount'))


# Agg_transaction

cursor.execute ("select * from agg_transaction")
connect.commit()
tabel2 = cursor.fetchall()

Agg_Transaction = pd.DataFrame(tabel2, columns=('State', 'Year', 'Quarter', 'Transaction_type', 'Transaction_count', 'Transaction_amount'))


# Agg_User

cursor.execute ("select * from agg_user")
connect.commit()
tabel3 = cursor.fetchall()

Agg_User = pd.DataFrame(tabel3, columns=('State', 'Year', 'Quarter', 'Brand', 'Transaction_count', 'Percentage'))


# Map_insurance

cursor.execute ("select * from map_insurance")
connect.commit()
tabel4 = cursor.fetchall()

Map_insurance = pd.DataFrame(tabel4, columns=('State', 'Year', 'Quarter', 'District', 'Transaction_count', 'Transaction_amount'))


# Map_transaction

cursor.execute ("select * from map_transaction")
connect.commit()
tabel5 = cursor.fetchall()

Map_Transaction = pd.DataFrame(tabel5, columns=('State', 'Year', 'Quarter', 'District', 'Transaction_count', 'Transaction_amount'))


# Map_User

cursor.execute ("select * from map_user")
connect.commit()
tabel6 = cursor.fetchall()

Map_User = pd.DataFrame(tabel6, columns=('State', 'Year', 'Quarter', 'District', 'RegisteredUsers', 'AppOpens'))


# Top_insurance

cursor.execute ("select * from top_insurance")
connect.commit()
tabel7 = cursor.fetchall()

Top_insurance = pd.DataFrame(tabel7, columns=('State', 'Year', 'Quarter', 'Pincode', 'Transaction_count', 'Transaction_amount'))


#Top_Transaction

cursor.execute('select * from top_transaction')
connect.commit()
tabel8 = cursor.fetchall()

Top_Transaction = pd.DataFrame(tabel8, columns=('State', 'Year', 'Quarter', 'Pincode', 'Transaction_count', 'Transaction_amount'))


#Top_User

cursor.execute('select * from top_user')
connect.commit()
tabel9 = cursor.fetchall()

Top_User = pd.DataFrame(tabel9, columns=('State', 'Year', 'Quarter', 'Pincode', 'RegisteredUsers'))

In [ ]:
# Sql Query

def top_chart_transaction_amount(table_name): 
    
    connect = psycopg2.connect(
            host = "localhost",
            user = 'postgres',
            port = '5432',
            password = '01234',
            database = 'phonepy_transaction'
    )

    connect.set_isolation_level (ISOLATION_LEVEL_AUTOCOMMIT)

    cursor = connect.cursor()

    # Plot_1

    query1 = f'''select state, Sum(transaction_amount) as transaction_amount
            from {table_name}
            group  by state
            order by transaction_amount DESC
            LIMIT 10;'''

    cursor.execute(query1)
    tabel_1 = cursor.fetchall()
    connect.commit()

    df_1 = pd.DataFrame(tabel_1, columns= ("state", "transaction_amount"))

    fig_amount_1 = px.bar(df_1, x='state', y='transaction_amount', title = 'Transaction Amount', 
                            color_discrete_sequence= px.colors.sequential.Aggrnyl, height=650, width=600)
    fig_amount_1.show()



    # Plot_2

    query2 = f'''select state, Sum(transaction_amount) as transaction_amount
                from {table_name}
                group  by state
                order by transaction_amount
                LIMIT 10;'''

    cursor.execute(query2)
    tabel_2 = cursor.fetchall()
    connect.commit()

    df_2 = pd.DataFrame(tabel_2, columns= ("state", "transaction_amount"))

    fig_amount_2 = px.bar(df_2, x='state', y='transaction_amount', title = 'Transaction Amount', 
                            color_discrete_sequence= px.colors.sequential.Aggrnyl_r, height=650, width=600)
    fig_amount_2.show()


    # Plot_3

    query3 = f'''select state, Avg(transaction_amount) as transaction_amount
                from {table_name}
                group  by state
                order by transaction_amount;'''

    cursor.execute(query3)
    tabel_3 = cursor.fetchall()
    connect.commit()

    df_2 = pd.DataFrame(tabel_3, columns= ("state", "transaction_amount"))

    fig_amount_3 = px.bar(df_2, y='state', x='transaction_amount', title = 'Transaction Amount', hover_name= "state", orientation= 'h', 
                            color_discrete_sequence= px.colors.sequential.Bluered_r, height=650, width=600)
    fig_amount_3.show()



In [3]:
# Transaction_Year_Based

def Transaction_amount_count_Y(df, year):
    #(Boolean Series)
    tacy = df[df['Quarter'] == year]
    tacy.reset_index(drop = True, inplace= True)

    tacyg = tacy.groupby('State')[['Transaction_count', 'Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)

    fig_amount = px.bar(tacyg, x='State', y='Transaction_amount', title =f'{year} Transaction Amount', 
                         color_discrete_sequence= px.colors.sequential.Aggrnyl, height=650, width=600)
    fig_amount.show()

    fig_count = px.bar(tacyg,x = 'State', y='Transaction_count', title =f'{year} Transaction Count',
                       color_discrete_sequence= px.colors.sequential.Aggrnyl, height=650, width=600)
    fig_count.show()

    url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    response = requests.get(url)

    data1 = json.loads(response.content)

    states_name = []

    for feature in data1["features"]:
        print(feature['properties'])
        states_name.append(feature['properties']['ST_NM'])

    states_name.sort()

    fig_india_1 = px.choropleth(tacyg, geojson=data1, locations='State', featureidkey= 'properties.ST_NM',  
                                color= 'Transaction_amount', color_continuous_scale= 'Rainbow', 
                                range_color= (tacyg ['Transaction_amount'].min(), tacyg ['Transaction_amount'].max()), 
                                hover_name= 'State', title= f"{year} TRANSACTION AMOUNT",  fitbounds="locations",
                                height=600, width=600 )
    
    fig_india_1.update_geos(visible= False)
    # st.plotly_chart(fig_india_1)
    fig_india_1.show()
    
    fig_india_2 = px.choropleth(tacyg, geojson=data1, locations='State', featureidkey= 'properties.ST_NM',  
                                color= 'Transaction_count', color_continuous_scale= 'Rainbow', 
                                range_color= (tacyg ['Transaction_count'].min(), tacyg ['Transaction_count'].max()), 
                                hover_name= 'State', title= f"{year} TRANSACTION COUNT",  fitbounds="locations",
                                height=600, width=600 )
    
    fig_india_2.update_geos(visible= False)
    # st.plotly_chart(fig_india_2)
    fig_india_2.show()

    return tacy

In [4]:
#Transaction_Quarter_Based

def Transaction_amount_count_Y_Q(df, quarter):
    #(Boolean Series)
    tacy = df[df['Quarter'] == quarter]
    tacy.reset_index(drop = True, inplace= True)

    tacyg = tacy.groupby('State')[['Transaction_count', 'Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)

    fig_amount = px.bar(tacyg, x='State', y='Transaction_amount', title =f"{tacy['Year'].min()} YEAR {quarter} QUARTER Transaction Amount", 
                         color_discrete_sequence= px.colors.sequential.Aggrnyl, height=650, width=600)
    fig_amount.show()

    fig_count = px.bar(tacyg,x = 'State', y='Transaction_count', title =f"{tacy['Year'].min()} YEAR {quarter} QUARTER Transaction Count",
                       color_discrete_sequence= px.colors.sequential.Aggrnyl, height=650, width=600)
    fig_count.show()

    url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    response = requests.get(url)

    data1 = json.loads(response.content)

    states_name = []

    for feature in data1["features"]:
        print(feature['properties'])
        states_name.append(feature['properties']['ST_NM'])

    states_name.sort()

    fig_india_1 = px.choropleth(tacyg, geojson=data1, locations='State', featureidkey= 'properties.ST_NM',  
                                color= 'Transaction_amount', color_continuous_scale= 'Rainbow', 
                                range_color= (tacyg ['Transaction_amount'].min(), tacyg ['Transaction_amount'].max()), 
                                hover_name= 'State', title= f"{tacy['Year'].min()} YEAR {quarter} QUARTER TRANSACTION AMOUNT",  fitbounds="locations",
                                height=600, width=600 )
    
    fig_india_1.update_geos(visible= False)
    # st.plotly_chart(fig_india_1)
    fig_india_1.show()
    
    fig_india_2 = px.choropleth(tacyg, geojson=data1, locations='State', featureidkey= 'properties.ST_NM',  
                                color= 'Transaction_count', color_continuous_scale= 'Rainbow', 
                                range_color= (tacyg ['Transaction_count'].min(), tacyg ['Transaction_count'].max()), 
                                hover_name= 'State', title= f"{tacy['Year'].min()} YEAR {quarter} QUARTER TRANSACTION COUNT",  fitbounds="locations",
                                height=600, width=600 )
    
    fig_india_2.update_geos(visible= False)
    # st.plotly_chart(fig_india_2)
    fig_india_2.show()

    return tacy


In [5]:
#Transaction_Type

def Agg_trans_Transaction_type(df,state):

    tacy = df[df['State'] == state]
    tacy.reset_index(drop = True, inplace= True)

    tacyg = tacy.groupby('Transaction_type')[['Transaction_count', 'Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)


    fig_pie_1 = px.pie(data_frame= tacyg, names= "Transaction_type", values= "Transaction_amount", 
                   width= 600, title= f"{state.upper()} TRANSATION_AMOUNT", hole= 0.5)
    fig_pie_1.show()

    fig_pie_2 = px.pie(data_frame= tacyg, names= "Transaction_type", values= "Transaction_count", 
                   width= 600, title= f"{state.upper()} TRANSATION_COUNT", hole= 0.5)
    fig_pie_2.show()

In [6]:
# Aggregated_User_analysis_1

def Agg_User_plot_1(df, year):
    aguy = df[df["Year"] == year]
    aguy.reset_index(drop = True, inplace= True)

    aguyg = pd.DataFrame (aguy.groupby("Brand")["Transaction_count"].sum())
    aguyg.reset_index(inplace= True)


    fig_bar_1 = px.bar(aguyg, x= "Brand", y ="Transaction_count", title= f"{year} BRAND and TRANSACTION_COUNT",
                    width=1000, color_discrete_sequence= px.colors.sequential.haline_r, hover_name= "Brand")
    fig_bar_1.show()

    return aguy

In [7]:
#Aggre_User_Analysis_2

def Agg_User_plot_2(df, quarter):

    aguyq = df[df["Quarter"] == quarter]
    aguyq.reset_index(drop = True, inplace= True)

    aguyqg = pd.DataFrame(aguyq.groupby("Brand")["Transaction_count"].sum())
    aguyqg.reset_index(inplace=True)

    fig_bar_1 = px.bar(aguyqg, x= "Brand", y ="Transaction_count", title= f"{quarter} QUARTER BRANDS and TRANSACTION_COUNT",
                        width=1000, color_discrete_sequence= px.colors.sequential.haline_r, hover_name= "Brand")
    fig_bar_1.show()

    return aguyq

In [8]:
#Aggre_User_Analysis_3

def Agg_User_plot_3 (df, state):
    auyqs = df[df["State"] == state]
    auyqs.reset_index (drop = True, inplace= True)

    fig_line_3 = px.line(auyqs, x = "Brand", y = "Transaction_count", hover_data= "Percentage",
                        title= f"{state.upper()} BRAND, TRANSACTION_COUNT, PERENTAGE", width=1000, markers= 'd')

    fig_line_3.show()

In [9]:
#Map_Insurance_District

def Map_insur_District(df,state):

    tacy = df[df['State'] == state]
    tacy.reset_index(drop = True, inplace= True)

    tacyg = tacy.groupby('District')[['Transaction_count', 'Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)

    
    fig_bar_1 = px.bar(tacyg, x= "Transaction_amount", y= "District", orientation= "h", height=600,
                       title= f"{state.upper()} DISTRICT AND TRANSACTION AMOUNT", color_discrete_sequence= px.colors.sequential.Mint_r)
    fig_bar_1.show()
    

    fig_bar_2 = px.bar(tacyg, x= "Transaction_count", y= "District", orientation= "h", height=600,
                       title= f"{state.upper()} DISTRICT AND TRANSACTION COUNT", color_discrete_sequence= px.colors.sequential.Bluered_r)
    fig_bar_2.show()


In [10]:
#Map_User_plot_1

def Map_User_plot_1(df, year):
    muy = df[df["Year"] == year]
    muy.reset_index(drop = True, inplace= True)

    muyg = muy.groupby("State") [["RegisteredUsers", "AppOpens"]].sum()
    muyg.reset_index(inplace=True)


    fig_line_1 = px.line(muyg, x="State", y=["RegisteredUsers", "AppOpens"], 
                        color_discrete_map={"RegisteredUsers": "#14129E", "AppOpens":"#ff7f0e"},
                        title=f"{year} REGISTERED USERS (vs) APP OPENS", width=1000, height=800, markers=True) 
    fig_line_1.show()

    return muy

In [11]:
#Map_User_plot_2

def Map_User_plot_2(df, quarter):
    muyq = df[df["Quarter"] == quarter]
    muyq.reset_index(drop = True, inplace= True)

    muyqg = muyq.groupby("State") [["RegisteredUsers", "AppOpens"]].sum()
    muyqg.reset_index(inplace=True)


    fig_line_1 = px.line(muyqg, x="State", y=["RegisteredUsers", "AppOpens"], 
                        color_discrete_map={"RegisteredUsers": "#12229E", "AppOpens":"#ff7f0e"},
                        title=f"{df['Year'].min()} YEAR {quarter} QUARTER REGISTERED USERS (vs) APP OPENS", width=1000, height=800, markers=True) 
    fig_line_1.show()

    return muyq

In [12]:
#Map_user_plot_3

def map_user_plot_3(df, state):

    muyqs = df[df["State"] == state]
    muyqs.reset_index(drop = True, inplace= True)

    fig_map_user_bar_1 = px.bar(muyqs, x = "RegisteredUsers", y = "District", orientation="h", 
                                title= "REGISTERED USERS", height= 800, color_discrete_sequence= px.colors.sequential.Rainbow_r)

    fig_map_user_bar_1.show()

    fig_map_user_bar_2 = px.bar(muyqs, x = "AppOpens", y = "District", orientation="h", 
                                title= "APPOPENS", height= 800, color_discrete_sequence= px.colors.sequential.Rainbow)

    fig_map_user_bar_2.show()

In [13]:
# Top_Insurance_plot_1

def Top_insurance_plot_1(df, state):
    
    tiy = df[df["State"] == state]
    tiy.reset_index(drop = True, inplace= True)

    fig_top_insur_bar_1 = px.bar(tiy, x = "Quarter", y = "Transaction_amount", hover_data="Pincode", #orientation="h",
                                    title= "TRANSACTION AMOUNT", height= 800, color_discrete_sequence= px.colors.sequential.Darkmint_r)

    fig_top_insur_bar_1.show()

    fig_top_insur_bar_2 = px.bar(tiy, x = "Quarter", y = "Transaction_amount", hover_data="Pincode", #orientation="h",
                                    title= "TRANSACTION COUNT", height= 800, color_discrete_sequence= px.colors.sequential.Agsunset_r)

    fig_top_insur_bar_2.show()

In [ ]:
#Top_User_plot_1

def Top_User_plot_1(df, year):
    tuy = df[df["Year"] == year]
    tuy.reset_index(drop = True, inplace= True)

    tuyg = pd.DataFrame (tuy.groupby(["State", "Quarter"])["RegisteredUsers"].sum())
    tuyg.reset_index(inplace= True)

    fig_top_plot_1 = px.bar(tuyg, x= "State", y= "RegisteredUsers", color= "Quarter", width=1000, height=800, 
                            color_discrete_sequence= px.colors.sequential.Burgyl, hover_name= "State",
                            title= f"{year} REGISTERED USER")
    fig_top_plot_1.show()

    return tuy

In [47]:
#Top_User_plot_2

def Top_User_plot_2(df, State):

    tuys = df[df ["State"]== State]
    tuys.reset_index(drop= True, inplace= True)

    fig_top_plot_2 = px.bar(tuys, x= "Quarter", y= "RegisteredUsers", hover_data= "Pincode",  width=1000, height=800, 
                                color_continuous_scale = px.colors.sequential.Magenta_r, 
                                title= "REGISTERED USER AND PINCODE QUARTER DATA")
    fig_top_plot_2.show()

# return tuy

## Aggregated_Insurance

In [ ]:
tac_Y = Transaction_amount_count_Y(Agg_insurance, 2018)

In [ ]:
Transaction_amount_count_Y_Q(tac_Y, 2)

## Aggregated_Transaction

In [ ]:
Agg_trans_tac_Y = Transaction_amount_count_Y(Agg_Transaction, 2022)

In [ ]:
Agg_trans_Transaction_type(Agg_trans_tac_Y,'Tamil Nadu')

In [ ]:
Agg_trans_tac_Y_Q = Transaction_amount_count_Y_Q(Agg_trans_tac_Y, 2)

In [ ]:
Agg_trans_Transaction_type(Agg_trans_tac_Y_Q,'Tamil Nadu')

In [19]:
Agg_trans_tac_Y_Q["State"].unique()

array([], dtype=object)

## Aggregated_USER

In [ ]:
Agg_User_Y = Agg_User_plot_1(Agg_User, 2018)

In [ ]:
Agg_User_Y_Q = Agg_User_plot_2(Agg_User_Y, 3)

In [ ]:
Agg_User_plot_3(Agg_User_Y_Q, 'Andaman & Nicobar')

## Map_Insurance

In [ ]:
Map_insur_tac_Y = Transaction_amount_count_Y(Map_insurance, 2022)

In [ ]:
Map_insur_District(Map_insur_tac_Y, 'Tamil Nadu')

In [ ]:
Map_insur_tac_Y_Q = Transaction_amount_count_Y_Q(Map_insur_tac_Y, 2)

In [ ]:
Map_insur_District(Map_insur_tac_Y_Q, 'Tamil Nadu')

## MAP_Transaction

In [ ]:
Map_trans_tac_Y = Transaction_amount_count_Y(Map_Transaction, 2023)

In [ ]:
Map_insur_District(Map_trans_tac_Y, 'Tamil Nadu')

In [ ]:
Map_trans_tac_Y_Q = Transaction_amount_count_Y_Q(Map_trans_tac_Y, 2)

In [ ]:
Map_insur_District(Map_trans_tac_Y_Q, 'Tamil Nadu')

## MAP_User

In [ ]:
Map_User_Y = Map_User_plot_1(Map_User, 2020)

In [ ]:
Map_User_Y_Q = Map_User_plot_2(Map_User_Y, 2)

In [ ]:
map_user_plot_3(Map_User_Y_Q, "Tamil Nadu")

## Top_Insurance

In [ ]:
Top_insur_tac_Y = Transaction_amount_count_Y(Top_insurance, 2020)

In [ ]:
Top_insurance_plot_1(Top_insur_tac_Y, "West Bengal")

In [ ]:
Top_insur_tac_Y_Q = Transaction_amount_count_Y_Q(Top_insur_tac_Y, 2)

## Top_Transaction

In [ ]:
Top_trans_tac_Y = Transaction_amount_count_Y(Top_Transaction, 2020)

In [ ]:
Top_insurance_plot_1(Top_trans_tac_Y, "West Bengal")

In [ ]:
Top_trans_tac_Y_Q = Transaction_amount_count_Y_Q(Top_trans_tac_Y, 2)

## Top User

In [ ]:
Top_User_Y = Top_User_plot_1(Top_User, 2021)

In [ ]:
Top_User_plot_2(Top_User_Y, "Tamil Nadu")

In [52]:
Top_User

,State,Year,Quarter,Pincode,RegisteredUsers
0,Andaman & Nicobar,2018,1,744103,1608
1,Andaman & Nicobar,2018,1,744101,1108
2,Andaman & Nicobar,2018,1,744105,1075
3,Andaman & Nicobar,2018,1,744102,1006
4,Andaman & Nicobar,2018,1,744104,272
...,...,...,...,...,...
9995,West Bengal,2024,4,733134,169596
9996,West Bengal,2024,4,700059,169448
9997,West Bengal,2024,4,711302,146034
9998,West Bengal,2024,4,700150,142151
